# init

In [1]:
from pymongo import MongoClient

client = MongoClient()
db = client.izidb

In [10]:
from scripts import izi
reload(izi)

n_topics = 100

--------------------> english word frequencies loaded


# get files list

In [21]:
root = u"../izi_data/"
filelist = izi.getFileList(root)
print len(filelist)
example =  filelist[0]
print example[len(root):]

195
1412 Standard English Review-en-ru-T.mxliff


# define functions

In [12]:
def insertDoc(path, root = root):
    document = dict()
    document['title'] = path[len(root):]
    full_text = izi.loadText(path)
    tokens = izi.tokenize( full_text)
    document['full_text'] = full_text
    document['tokens'] = tokens
    topics =  izi.topicsFromTokens(izi.tokenize(full_text))
    semantic_vec = [0.] * n_topics
    for i in topics:
        semantic_vec[i[0]] = i[1]
    document['semantic_vec'] = semantic_vec
    current_id = db.documents.save(document)
    # create links
    cursor = db.documents.find()
    for doc in cursor:
    	y = doc['semantic_vec']
    	y_id = doc["_id"]
    	if y_id != current_id:
	    	s = izi.getSimilarity( semantic_vec, y)
	    	db.similarities.insert({'source': current_id , 'target': y_id, 'value': s})

# Brown

In [13]:
root2 = u"../brown/"
import nltk
brown_ids = nltk.corpus.brown.fileids()
for i in brown_ids:
    f = open( root2 + i + ".txt", "w")
    f.write(nltk.corpus.brown.raw(i) )

In [20]:
for f in izi.getFileList(root2):
    insertDoc(f, root = root2)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:13: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:21: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


KeyboardInterrupt: 

In [21]:
# cursor = db.documents.find_one({ "_id": id})
# for document in cursor:
#     print document

# add all documents

In [ ]:
fails = []
i = 0
maxx = 500
for f in filelist:
    if i < maxx:
        try:
            insertDoc(f)
            print i,
        except:
            fails.append(f)
            print "#",

        i += 1
    
print
print
print "%s fails" %len(fails)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:13: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:21: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 # # 

In [8]:
for f in fails:
    print f

../izi_data/Beth�s Blog 1 v3-en-da-T.mxliff
../izi_data/Beth�s Blog 2 v1-en-da-T.mxliff
../izi_data/Beth�s Blog 3 v3-en-da-T.mxliff
../izi_data/Hvad er mikrogr�nt_en-GB-en-it-T.mxliff
../izi_data/Personlig s�tning_en-GB-en-it-T.mxliff
../izi_data/Sprouts v�rdier_en-GB-en-it-T.mxliff
../izi_data/S�DAN G�R DU _en-GB-en-it-T.mxliff
../izi_data/Turkish_Basketball_Clubs_Evaluationprogramme15-16_Repucom_150612_�bersetzten-en-tr-T.mxliff
../izi_data/Underbilag 3 3 Leverand�rens generelle l�sningsbeskrivelse - BI-en-da-T.mxliff
../izi_data/Underbilag 3 3 Leverand�rens generelle l�sningsbeskrivelse - BU-en-da-T.mxliff
../izi_data/Underbilag 3 3 Leverand�rens generelle l�sningsbeskrivelse - IN-en-da-T.mxliff
../izi_data/Underbilag 3 3 Leverand�rens generelle l�sningsbeskrivelse - OP-en-da-T.mxliff
../izi_data/Underbilag 3 3 Leverand�rens generelle l�sningsbeskrivelse - PS-en-da-T.mxliff
../izi_data/Underbilag 3 3 Leverand�rens generelle l�sningsbeskrivelse - RE-en-da-T.mxliff
../izi_data/Underbi

# tests

In [ ]:
cursor = db.documents.find_one()
for document in cursor:
    print document
    
    
x = cursor["_id"]
print str(x)

db.documents.find_one({ "_id": x})["_id"]

In [ ]:
db.similarities.find_one()

# reset collections

In [22]:
db.restaurants.drop()
db.documents.drop()
db.similarities.drop()